**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from scipy import stats
from scipy.sparse import issparse
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

In [ ]:
class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors

    def my_kneighbors(self, X, n_neighbors=None):
        if n_neighbors is None:
            n_neighbors = self.n_neighbors
        shift = 100
        x_dist = (X ** 2).sum(axis=1)
        Y = self._fit_X[0: shift, :]
        y_dist = (Y ** 2).sum(axis=1)
        xy_dist = X.dot(Y.T)
        dist = y_dist - 2 * xy_dist + x_dist.reshape(-1, 1)
        old_neighbors_ind = np.argsort(np.sqrt(dist), axis=1)[:, :n_neighbors]
        start_shift = shift
        while (start_shift < self._fit_X.shape[0]):
            Y = self._fit_X[start_shift:start_shift + shift, :]
            y_dist = (Y ** 2).sum(axis=1)
            xy_dist = X.dot(Y.T)
            dist = y_dist - 2 * xy_dist + x_dist.reshape(-1, 1)

            new_neighbors_ind = np.argsort(np.sqrt(dist), axis=1)[:, :n_neighbors] + start_shift
            old_neighbors_ind = np.concatenate((old_neighbors_ind, new_neighbors_ind), axis=1)
            start_shift += shift

        if (start_shift > shift):
            neighbors = np.empty(((X.shape[0], n_neighbors)), dtype=int)
            for i, row in enumerate(X):
                x_dist = (row ** 2).sum()
                Y = self._fit_X[old_neighbors_ind[i], :]
                y_dist = (Y ** 2).sum(axis=1)
                xy_dist = row.dot(Y.T)
                dist = y_dist - 2 * xy_dist + x_dist.reshape(-1, 1)
                n = np.argsort(np.sqrt(dist), axis=1)[:, :n_neighbors]

                neighbors[i] = old_neighbors_ind[i][n]
        else:
            neighbors = old_neighbors_ind
        return neighbors

    def fit(self, X, y):
        self.classes_, _ = np.unique(y, return_inverse=True)
        self._fit_X = X
        self._y = np.array(y, dtype=y.dtype)
        return self

    def predict(self, X):
        neigh_ind = self.my_kneighbors(X)
        classes_ = self.classes_
        _y = self._y
        n_samples = X.shape[0]
        y_pred = np.empty(n_samples, dtype=classes_[0].dtype)
        mode, _ = stats.mode(_y[neigh_ind], axis=1)
        y_pred = mode
        return y_pred.ravel()

    def predict_proba(self, X):
        neigh_ind = self.my_kneighbors(X)
        classes_ = self.classes_
        _y = self._y
        n_samples = X.shape[0]
        weights = np.ones_like(neigh_ind, dtype=np.int)
        all_rows = np.arange(X.shape[0])
        pred_labels = np.array(_y[neigh_ind], dtype=np.int)
        proba = np.zeros((n_samples, classes_.size))
        for i, idx in enumerate(pred_labels.T):
            proba[all_rows, idx] += weights[:, i]
        return proba / weights.shape[1]

    def score(self, X, y):
        y_pred = self.predict(X)
        score = y == y_pred
        return np.sum(score) / score.shape[0]

**IRIS**

In [ ]:
iris = datasets.load_iris()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

In [ ]:
# Задание 3

In [ ]:
# Построение графика
from sklearn.metrics import accuracy_score

neighbor = range(1, 125)
errors_train = []
errors_test = []
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)
for k in neighbor:
    my_clf = MyKNeighborsClassifier(n_neighbors=k)
    my_clf.fit(X_train, y_train)
    predict_test = my_clf.predict(X_test)
    predict_train = my_clf.predict(X_train)
    errors_train.append(accuracy_score(predict_train, y_train))
    errors_test.append(accuracy_score(predict_test, y_test))

plt.plot(neighbor, errors_train, color='blue', label='train')
plt.plot(neighbor, errors_test, color='red', label='test')
plt.legend()

In [ ]:
# нахождение наилучшего k
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

param_grid = {
    "mykneighborsclassifier__n_neighbors": range(1, 125)
}

reg = make_pipeline(
    StandardScaler(),
    MyKNeighborsClassifier()
)

**MNIST**

In [ ]:
mnist = fetch_mldata('MNIST original')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, stratify=mnist.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn
# Это библиотека визуализации Python на основе matplotlib. 
# Обеспечивает высокоуровневый интерфейс для рисования  статистических графиков.

# matplotlib
# библиотека для построения научных графиков(линейных диаграмм, гистограмм, диаграмм разбросаи т.д) в Python

# train_test_split
# Разделяет np массив на два: train и test, на первом происходит обучение и тестирование на втором соответственно

# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# Класс-контейнер инкапсулирует трансформаторы и предиктора внутри себя,
# и предоставляющий единый интерфейс для работы

# StandardScaler (from sklearn.preprocessing import StandardScaler)
# Класс преобразует данные таким образом, что их распределение будет иметь матожидание 0
# и дисперсия 1.

# ClassifierMixin
# Базовый Класс Mixin для всех классификаторов в scikit-learn,
# содержащий метод score(возвращает среднюю точность данных данных)

# NeighborsBase 
# Базовый Класс, для оценки ближайших соседей
# 
# KNeighborsMixin
# Базовый Класс Mixin для  для поиска k-соседей

# SupervisedIntegerMixin
# Базовый Класс Mixin проверяет параметры и вызывает реальную функцию.